In [1]:
# read it in to inspect it
with open('Data\\input.txt', 'r', encoding='utf-8') as f:
    text = f.read()


In [2]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
text[0:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(chars[16]+chars[1]+chars[16]+chars[0]+chars[2])


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
D D
!


In [5]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [6]:
import torch 
#wrap the data in a tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])
torch.layout

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

torch.layout

In [7]:
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [8]:
decode([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61])

'First Citizen:\nBefore w'

In [9]:
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
x= train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target= y[t]
    print(f"when context is {context}, target is {target}")

when context is tensor([18]), target is 47
when context is tensor([18, 47]), target is 56
when context is tensor([18, 47, 56]), target is 57
when context is tensor([18, 47, 56, 57]), target is 58
when context is tensor([18, 47, 56, 57, 58]), target is 1
when context is tensor([18, 47, 56, 57, 58,  1]), target is 15
when context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
when context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [11]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?
ix = torch.randint(len(data) - block_size, (batch_size,1))
ix

tensor([[1078327],
        [ 453969],
        [  41646],
        [ 671252]])

In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    xappender = []
    yappender = []
    for i in ix:
        xappender.append(data[i:i+block_size])
    x = torch.stack(xappender)
    for i in ix:
        yappender.append(data[i+1:i+block_size+1])
    y = torch.stack(yappender)

    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [13]:
xb, yb

(tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [14]:
import torch.nn as nn
torch.manual_seed(1337)
embedding_table = torch.randint(1,10,(6,5))
print(embedding_table, "\n\n")
a = torch.tensor([[1,1,1,1],[3,3,3,3]])
print("this is a tensor\n", a, "\n\n")
embedding_table[a]

tensor([[2, 5, 6, 8, 9],
        [8, 3, 9, 1, 9],
        [3, 7, 1, 8, 4],
        [2, 9, 2, 1, 2],
        [9, 1, 9, 1, 7],
        [7, 4, 7, 3, 9]]) 


this is a tensor
 tensor([[1, 1, 1, 1],
        [3, 3, 3, 3]]) 




tensor([[[8, 3, 9, 1, 9],
         [8, 3, 9, 1, 9],
         [8, 3, 9, 1, 9],
         [8, 3, 9, 1, 9]],

        [[2, 9, 2, 1, 2],
         [2, 9, 2, 1, 2],
         [2, 9, 2, 1, 2],
         [2, 9, 2, 1, 2]]])

In [10]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_emebedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets):
        logits = self.token_emebedding_table(idx)

        return logits

m = BigramLanguageModel(vocab_size)
out = m(xb, yb)
print(out.shape)


torch.Size([4, 8, 65])


In [11]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)

logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)


tensor([[ 0, 31, 56, 12, 55, 28,  7, 29, 35, 49, 58, 36, 53, 24,  4, 48, 24, 16,
         22, 45, 27, 24, 34, 64,  5, 30, 21, 53, 16, 55, 20, 42, 46, 57, 34,  4,
         60, 24, 24, 62, 39, 58, 48, 57, 41, 25, 54, 61, 24, 17, 30, 31, 28, 63,
         39, 53,  8, 55, 44, 64, 57,  3, 37, 57,  3, 64, 18,  7, 61,  6, 11, 43,
         17, 49, 64, 62, 48, 45, 15, 23, 18, 15, 46, 57,  2, 47, 35, 35,  8, 27,
         40, 64, 16, 52, 62, 13,  1, 25, 57,  3,  9]])

True

In [83]:
optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-03)

for i in range(10000):
    xb, yb = get_batch('train')
    _, loss = m(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print(loss)


tensor(4.6595, grad_fn=<NllLossBackward0>)
tensor(4.8696, grad_fn=<NllLossBackward0>)
tensor(4.9960, grad_fn=<NllLossBackward0>)
tensor(4.4939, grad_fn=<NllLossBackward0>)
tensor(4.7628, grad_fn=<NllLossBackward0>)
tensor(4.8557, grad_fn=<NllLossBackward0>)
tensor(4.5393, grad_fn=<NllLossBackward0>)
tensor(4.6646, grad_fn=<NllLossBackward0>)
tensor(4.6165, grad_fn=<NllLossBackward0>)
tensor(4.8863, grad_fn=<NllLossBackward0>)
tensor(4.4042, grad_fn=<NllLossBackward0>)
tensor(5.0736, grad_fn=<NllLossBackward0>)
tensor(4.3425, grad_fn=<NllLossBackward0>)
tensor(4.5322, grad_fn=<NllLossBackward0>)
tensor(4.7616, grad_fn=<NllLossBackward0>)
tensor(5.2401, grad_fn=<NllLossBackward0>)
tensor(4.3989, grad_fn=<NllLossBackward0>)
tensor(4.8298, grad_fn=<NllLossBackward0>)
tensor(4.6245, grad_fn=<NllLossBackward0>)
tensor(4.6013, grad_fn=<NllLossBackward0>)
tensor(4.6702, grad_fn=<NllLossBackward0>)
tensor(4.4811, grad_fn=<NllLossBackward0>)
tensor(4.6186, grad_fn=<NllLossBackward0>)
tensor(4.52

In [86]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=1000)[0].tolist()))


Fous anowiu thelot!
NGHe t,
c; sorer, sthan atiky ser dsprd SP llk'sshe.
Ply, pleave ake thou t ay in fis; dalve s, amen,
Fon? st; he ath third we.
Mou rene therdus alivinknthe, t kpind,
YWIEJWII su s m iasknd, Ywchetecabrid froffewhs k' IAUCEdrle ve IsbaxEEJBE$TVHAhilu my t
Dd, l;
IN:
are a To come ngCo ncereroe lousigCOK:
DZMPy ddgllthishind rocothavom!
Thedegnk araste t ozez'rlinglll.
War sell, cand:

Whidore of.
til!Zy ajit!
lorot hed d wan n weew;
NCTor pind;AThigsthieakerartel dimeal!
AU3QYQLI hand ar benoocothrh meve iOME:
R od se;
Byar MSPeank k alin me blor irist yocoongend III'daurehe jx,
C
In Cladithayenardithal d thart:

f.

x aUpt e y s arer funerured, ldaty; t isthou.
As brepand IERing?uncengonilyerd, mo thexrand HENFouOLERIADult gak!
RReee:
Putomy chirs
UEMPyeten y pooanowngu, ath3QQJandstha-wo storovier'd, thaingispie indpat;


WAs y, bicarom d COLINan

IOul'mbathy mbas t toar, obed st, p'shat palves.
The, d qKprevy' t d t te NCIN.
ANI ul s' dit m e g'wif!
WIingl me th

In [29]:
import torch.nn.functional as F
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)
tril = torch.tril(torch.ones(T,T))
print(tril)

wei = torch.zeros((T,T)) #this is supposedly the result of the Q*K^T which is the attention weights before softmax and right now it is just initizliaed as 0s. Which is not right. We will change this
wei = wei.masked_fill(tril==0, float('-inf'))
print("Masked Fill of tril with -inf. Notice how the weights are set to zero for attention\n", wei)
wei = F.softmax(wei, dim=-1)
print(wei)
out = wei @ x
print(out.shape)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
Masked Fill of tril with -inf. Notice how the weights are set to zero for attention
 tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.00

# Now towards really making the wei matrix with actual query and keys matrics

In [49]:
import torch.nn.functional as F
torch.manual_seed(1337)
B,T,C = 4,8,32
x = torch.randn(B,T,C)

#single attention head with headsize 16
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x) #(B,T,16)
q = query(x) #(B,T,16)
wei = q @ k.transpose(-2,-1) #k.transpose(-2,-1) is going to swap the dimension of the last 2 dims
#this transpose will lead to this shape --> B,16,T, and remember that 16 is the head size.
#and since q is of shape B,T,16 we can matrix multiply these. giving a shape of B,T,T,
# this T,T will become our attention weights and its easy to see since it is T,T how each token interacts
#with other tokens.


tril = torch.tril(torch.ones(T,T))
print(tril)
#we don't need this line anymore, we have now computed our initial wei with queries and keys
# wei = torch.zeros((T,T)) #this is supposedly the result of the Q*K^T which is the attention weights before softmax and right now it is just initizliaed as 0s. Which is not right. We will change this

#we are still going to use the tril matrix we created to mask out the attention between future tokens in our
wei = wei.masked_fill(tril==0, float('-inf'))
print("Masked Fill of tril with -inf. Notice how the weights are set to zero for attention\n", wei)
wei = F.softmax(wei, dim=-1)
print(wei)
v = value(x)
out = wei @ v
print(out.shape)

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
Masked Fill of tril with -inf. Notice how the weights are set to zero for attention
 tensor([[[-1.7629e+00,        -inf,        -inf,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [-3.3334e+00, -1.6556e+00,        -inf,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [-1.0226e+00, -1.2606e+00,  7.6228e-02,        -inf,        -inf,
                 -inf,        -inf,        -inf],
         [ 7.8359e-01, -8.0143e-01, -3.3680e-01, -8.4963e-01,        -inf,
                 -inf,        -inf,        -inf],
         [-1.2566e+00,  1.8719e-02, -7.8797e-01, -1.3204e+00,  2.0363e+00,
   

In [48]:
wei

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [45]:
k.shape

torch.Size([4, 8, 16])